In [ ]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-afr_6ncb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-afr_6ncb
  Resolved https://github.com/huggingface/transformers.git to commit e0c3cee17085914bbe505c159beeb8ae39bc37dd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor,AutoFeatureExtractor, Wav2Vec2BertModel,AutoProcessor,AutoModelForCTC,Wav2Vec2BertProcessor
import torch
model_w2v = AutoModelForCTC.from_pretrained("Cafet/w2v-bert-version-final").to("cuda")
processor_w2v = Wav2Vec2BertProcessor.from_pretrained("Cafet/w2v-bert-version-final")
processor_tiny = WhisperProcessor.from_pretrained("Cafet/whisper-tiny-final")
model_tiny = WhisperForConditionalGeneration.from_pretrained("Cafet/whisper-tiny-final").to("cuda")
precessor_large = WhisperProcessor.from_pretrained("Cafet/whisper-large-final")
model_large = WhisperForConditionalGeneration.from_pretrained("Cafet/whisper-large-final").to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict, concatenate_datasets, Audio
cv = load_dataset('Cafet/mas_ex', split='train[:12%]', use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [ ]:
# cv = cv.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes","variant"])
cv

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 52
})

In [ ]:
cv = cv.cast_column('audio', Audio(sampling_rate=16000))
import re
chars_to_remove_regex = '[\,\?\.\!\-\–\$\;\:\"\“\%\‘\”\]\[\�\'\»\0\1\2\3\4\'̆\'\5\6\7\«\¾\+\0\^\xa0\~\½\²\u200b\’\&\/\õ]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()

    return batch
nums_to_remove_regex = '[1234567890\[\]]'

def remove_numbers_and_brackets(batch):
    batch["transcription"] = re.sub(nums_to_remove_regex, '', batch["transcription"]).lower()
    return batch

def remove_latin_characters(batch):
    batch["transcription"] = re.sub(r'[a-z]+', '', batch["transcription"])
    return batch

eval_data = cv.map(remove_special_characters)
eval_data = eval_data.map(remove_numbers_and_brackets)
eval_data = eval_data.map(remove_latin_characters)

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = processor_w2v(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["input_length"] = len(batch["input_features"])

    # batch["sentence"] = processor_w2v(text=batch["sentence"]).input_ids
    return batch

In [ ]:
data = eval_data.map(prepare_dataset)

In [ ]:
data

Dataset({
    features: ['audio', 'transcription', 'input_features', 'input_length'],
    num_rows: 622
})

In [ ]:
def get_audio_duration(audio_data, sampling_rate):
    duration_seconds = len(audio_data) / sampling_rate
    return duration_seconds / 60
total_duration_minutes = sum(get_audio_duration(entry["audio"]["array"], entry["audio"]["sampling_rate"]) for entry in data)

print("Total length of audio files in minutes:", total_duration_minutes)

Total length of audio files in minutes: 85.69120416666661


In [ ]:
import torch
def tiny(batch):
    audio = batch["audio"]
    input_features = processor_tiny(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    batch["reference"] = processor_tiny.tokenizer._normalize(batch['transcription'])

    with torch.no_grad():
        predicted_ids = model_tiny.generate(input_features.to("cuda"))[0]
    transcription = processor_tiny.decode(predicted_ids)
    batch["prediction_whisper_tiny"] = processor_tiny.tokenizer._normalize(transcription)
    return batch
def w2v(batch):
    input_features = torch.tensor(batch["input_features"]).to("cuda")
    logits = model_w2v(input_features.unsqueeze(0)).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]
    transcription = processor_w2v.decode(pred_ids)
    batch["prediction_w2v_test"] = transcription
    return batch

def large(batch):
    audio = batch["audio"]
    input_features = processor_large(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features

    with torch.no_grad():
        predicted_ids = model_large.generate(input_features.to("cuda"))[0]
    transcription = processor_large.decode(predicted_ids)
    batch["prediction_whisper_large"] = processor_large.tokenizer._normalize(transcription)
    return torch.batch_norm_backward_elemt


In [ ]:
inference = data.map(tiny)
inference = inference.map(w2v)
inference = inference.map(large)

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/tokenization_whisper.py:500: UserWarning: The private method `_normalize` is deprecated and will be removed in v5 of Transformers.You can normalize an input string using the Whisper English normalizer using the `normalize` method.
  warnings.warn(


Map:   0%|          | 0/622 [00:00<?, ? examples/s]

In [ ]:
!pip install evaluate
!pip install jiwer

In [ ]:
from evaluate import load

wer = load("wer")
cer = load("cer")
print('w2v wer: ',100 * wer.compute(references=inference["reference"], predictions=inference["prediction_w2v_test"]))
print('whisper_tiny wer: ',100 * wer.compute(references=inference["reference"], predictions=inference["prediction_whisper_tiny"]))
print('w2v cer: ',100 * cer.compute(references=inference["transcription"], predictions=inference["prediction_w2v_test"]))
print('whisper_tiny cer: ',100 * cer.compute(references=inference["transcription"], predictions=inference["prediction_whisper_tiny"]))
print('w2v wer: ',100 * wer.compute(references=inference["reference"], predictions=inference["prediction_whisper_large"]))
print('whisper_tiny wer: ',100 * wer.compute(references=inference["reference"], predictions=inference["prediction_whisper_large"]))

w2v wer:  43.964661504864004
whisper_tiny wer:  83.89914631725233
w2v cer:  16.975061488302924
whisper_tiny cer:  34.15317737230452


In [ ]:
inference.push_to_hub('inference', split = 'validation')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Cafet/inference/commit/7ff2516069dd160d22e03e5ec517fd9db836fab3', commit_message='Upload dataset', commit_description='', oid='7ff2516069dd160d22e03e5ec517fd9db836fab3', pr_url=None, pr_revision=None, pr_num=None)